
# Outpatient Resource Allocation and Turnaround Time – Toy Dataset

This notebook defines and documents a synthetic planning problem for an outpatient treatment unit (e.g., chemotherapy / infusion).
It is designed to test the following capabilities of our problem::

- continuous **resource variables** (staff allocations),
- **derived performance variables** (average turnaround time, TAT),
- **binary configuration decisions** (template selection, overflow capacity),
- multiple, potentially conflicting **goals** (TAT, cost, access).

The structure extends beyond basic nutrition or simple waiting-time examples by introducing discrete choices and
resource–performance linkages that resemble appointment scheduling and resource allocation models in the health operations literature.

## Literature Sources:

https://onlinelibrary.wiley.com/doi/10.1155/2013/680152 
https://www.sciencedirect.com/science/article/pii/S2211692316301291 
https://or.stackexchange.com/questions/6454/need-help-with-an-appointment-scheduling-problem https://repositorio.uac.pt/bitstreams/5edbaa77-2aa3-40e8-9022-4345df22f7e3/download

The dataset below is synthetically generated with GPT as none of the papers actually had dataset released or something


In [ ]:

import pandas as pd

# ---------------------------------------------
# 1. Service units inside the outpatient system
# ---------------------------------------------

units = pd.DataFrame({
    "Unit": ["Triage", "Consultation", "Infusion"],
    "Staff_Type": ["Nurse", "Doctor", "Nurse"],
    # Feasible staff bounds for a single planning day
    "Min_Staff": [1, 1, 2],
    "Max_Staff": [3, 4, 5],
    # Cost per staff-hour in arbitrary units
    "Staff_Cost_per_Hour": [20.0, 80.0, 25.0],
})

# ---------------------------------------------
# 2. Patient classes
# ---------------------------------------------
# Each class follows the same Triage → Consultation → Infusion structure,
# but with different volumes and baseline TAT.

patient_classes = pd.DataFrame({
    "Class": ["Chemo_Long", "Chemo_Short", "Followup"],
    # Expected arrivals per planning day
    "Daily_Volume": [10, 20, 30],
    # Baseline average turnaround time (minutes) under Min_Staff
    "Base_TAT": [360.0, 240.0, 120.0],
    # Service-level targets on TAT (minutes)
    "TAT_Target": [270.0, 180.0, 90.0],
    # Priority weights to reflect clinical / managerial importance
    "Priority_Weight": [3.0, 2.0, 1.0],
})

# ---------------------------------------------
# 3. TAT sensitivity to staffing by unit
# ---------------------------------------------
# Linear surrogate: for each extra staff member above Min_Staff,
# TAT for class c is reduced by Gamma minutes at that unit.

tat_sensitivity = pd.DataFrame({
    "Class": ["Chemo_Long"] * 3 + ["Chemo_Short"] * 3 + ["Followup"] * 3,
    "Unit":  ["Triage", "Consultation", "Infusion"] * 3,
    # Minutes of TAT reduction per extra staff member in that unit
    "Gamma": [
        3.0, 10.0, 12.0,   # Chemo_Long
        1.0, 6.0,  6.0,    # Chemo_Short
        0.5, 3.0,  2.0,    # Followup
    ],
})

# ---------------------------------------------
# 4. Discrete configuration options (for binary decisions)
# ---------------------------------------------
# These are "bundle" decisions that should be modelled with binary variables:
#   - Exactly one infusion scheduling template must be chosen.
#   - Overflow beds can optionally be opened.

config_options = pd.DataFrame({
    "Option_ID": [
        "INF_TEMPLATE_BASE",
        "INF_TEMPLATE_EXTENDED",
        "INF_OVERFLOW_BEDS",
    ],
    # Type groups options that compete or interact
    "Option_Type": [
        "Infusion_Template",
        "Infusion_Template",
        "Capacity_AddOn",
    ],
    "Description": [
        "Baseline infusion day: 8-hour day, standard chair schedule.",
        "Extended infusion day: 10-hour day, more evenly spread starts.",
        "Open 2 overflow infusion beds for the day.",
    ],
    # Additional nurse-hours required by this option
    "Extra_Nurse_Hours": [
        0.0,   # baseline
        8.0,   # one extra nurse for 8 hours
        4.0,   # additional monitoring time for overflow beds
    ],
    # Additional fixed cost of activating this option (per day)
    "Fixed_Cost": [
        0.0,
        400.0,
        200.0,
    ],
    # Approximate multiplicative factor on effective infusion capacity
    "Infusion_Capacity_Factor": [
        1.00,   # baseline
        1.25,
        1.10,
    ],
})

# ---------------------------------------------
# 5. Global planning parameters
# ---------------------------------------------

global_params = {
    # Baseline planning horizon in hours
    "Base_Hours_Per_Day": 8.0,
    # Budget cap for staff cost + fixed option costs (optional)
    "Max_Daily_Budget": 6000.0,
    # Minimum fraction of each class that should be treated in a day
    # (can be used for access / fairness constraints)
    "Min_Service_Prop": {
        "Chemo_Long": 0.6,
        "Chemo_Short": 0.5,
        "Followup": 0.4,
    },
}

units, patient_classes, tat_sensitivity, config_options, global_params



## Data Structures and Column Descriptions

### `units` – service stages and staffing parameters

Columns:

- **`Unit`**  
  Logical stage of the outpatient flow. All patient classes are assumed to pass through all units in sequence (e.g., Triage → Consultation → Infusion).

- **`Staff_Type`**  
  Dominant staff category driving capacity at the unit (e.g., nurses in triage and infusion, physicians in consultation). Used for cost accounting and possible skill-mix extensions.

- **`Min_Staff`, `Max_Staff`**  
  Lower and upper bounds on the number of staff assigned to the unit for the planning day. These induce box constraints on staffing decision variables.

- **`Staff_Cost_per_Hour`**  
  Cost per staff-hour at that unit, used to construct daily staffing cost expressions such as.

---

### `patient_classes` – demand and TAT targets

Columns:

- **`Class`**  
  Aggregate patient category (e.g., long vs short chemotherapy protocols, follow-up visits).

- **`Daily_Volume`**  
  Expected number of arrivals of that class during the planning day.

- **`Base_TAT`**  
  Baseline average turnaround time (minutes) for the class when each unit operates at its `Min_Staff` level. Used as a reference in the linear TAT approximation.

- **`TAT_Target`**  
  Service-level target for average TAT (minutes) for the class. Natural target for TAT-related GLP goals.

- **`Priority_Weight`**  
  Relative importance of meeting the TAT target for this class. Can be used as the weight in a goal on `TAT_c` or in a multi-objective objective function.

---

### `tat_sensitivity` – linear TAT reduction per extra staff

Columns:

- **`Class`** / **`Unit`**  
  Identify the class–unit pair for which a sensitivity parameter is specified.

- **`Gamma`**  
  Minutes of TAT reduction per additional staff member at unit for class, relative to `Min_Staff`.  
  A typical surrogate is:  
This provides a deterministic linear proxy for the queueing effects described in resource-allocation and simulation-based appointment scheduling papers.

---

### `config_options` – discrete configuration options

Columns:

- **`Option_ID`**  
  Identifier for a configuration option. Intended to map one-to-one with a binary decision variable \(z_k \in \{0,1\}\).

- **`Option_Type`**    
Logical grouping of mutually exclusive options. For Infusion_Template, exactly one template should be selected; Capacity_AddOn options can be modelled as independent on/off decisions.

- **`Extra_Nurse_Hours`**  
  Additional nurse-hours required if the option is activated.

- **`Fixed_Cost`**  
  Fixed daily cost of activating the option (e.g., overheads for extended opening hours or using overflow beds). Introduces fixed-charge effects.

- **`Infusion_Capacity_Factor`**  
  Multiplicative factor on effective infusion capacity when the option is active. For instance, a factor of 1.25 implies a 25% increase in infusion capacity relative to the baseline.

---

### `global_params` – global planning parameters

Keys:

- **`Base_Hours_Per_Day`**  
  Baseline planning horizon in hours, used to convert staff counts at each unit into staff-hours for cost calculations.

- **`Max_Daily_Budget`**  
  Optional budget cap for total daily cost (staffing plus fixed configuration costs).

- **`Min_Service_Prop`**  
  Dictionary mapping each class to a minimum fraction of its daily volume that should be treated.

In [ ]:

print("Units:")
display(units)
print("\nPatient classes:")
display(patient_classes)
print("\nTAT sensitivity:")
display(tat_sensitivity)
print("\nConfiguration options:")
display(config_options)
print("\nGlobal parameters:")
display(global_params)
